In [17]:
from pynq import Overlay
from pynq import MMIO
import time

design = Overlay("./Lab3.bit")

In [19]:
cdma = MMIO(0x7E200000, 0x10000) # Range = 0x10000 = 2^16 = 64KB

ddr = MMIO(0x10000000, 0x4000) # Range = 0x4000 = 2^14 = 16KB

bram = MMIO(0xC0000000, 0x4000) # Range = 0x4000 = 2^14 = 16KB

# ----------------------------
# STEP 1: ARM cpu write data to DDR
a = 40
b = 40
ddr.write(0x00, a)         # Write a to 0x30000000
ddr.write(0x04, b)         # Write b to 0x30000004

# ----------------------------
# STEP 2: CDMA transfers data from DDR to BRAM
cdma.write(0x00, 0x04)             # Start CDMA
cdma.write(0x18, 0x10000000)       # Source: DDR
cdma.write(0x20, 0xC0000000)       # Dest: BRAM
cdma.write(0x28, 0x08)             # BTT = 8 bytes (2 word)

time.sleep(0.01)                   # Wait for CDMA transfer to complete

# ----------------------------
# STEP 3: CDMA transfers data from BRAM back to DDR (return result)
cdma.write(0x00, 0x04)             # Start CDMA transfer
cdma.write(0x18, 0xC000000C)       # Source address: BRAM
cdma.write(0x20, 0x1000000C)       # Destination address: DDR
cdma.write(0x28, 0x10)             # BTT = 16 bytes (4 words)

time.sleep(0.01)                   # Wait for CDMA transfer to complete

# ----------------------------
# STEP 4: Read the result from DDR (at offset 0x0C)
result = ddr.read(0x0C)
print(f"ans of {a} + {b} = {result}")

ans of 40 + 40 = 80
